In [0]:
%pip install nltk
%pip install mlflow

Python interpreter will be restarted.
Collecting nltk
 Downloading nltk-3.6.1-py3-none-any.whl (1.5 MB)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.8/site-packages (from nltk) (0.17.0)
Collecting tqdm
 Downloading tqdm-4.60.0-py2.py3-none-any.whl (75 kB)
Collecting click
 Downloading click-7.1.2-py2.py3-none-any.whl (82 kB)
Collecting regex
 Downloading regex-2021.4.4-cp38-cp38-manylinux2014_x86_64.whl (733 kB)
Installing collected packages: tqdm, click, regex, nltk
Successfully installed click-7.1.2 nltk-3.6.1 regex-2021.4.4 tqdm-4.60.0
WARNING: You are using pip version 20.2.4; however, version 21.0.1 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-29616419-2237-4cad-81e8-dd77627eec76/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting mlflow
 Downloading mlflow-1.15.0-py3-none-any.whl (14.2 MB)
Requirement already satisfied: requests>=2.17.3 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2.24.0)
Collecting Flask
 Downloading Flask-1.1.2-py2.py3-none-any.whl (94 kB)
Collecting sqlalchemy
 Downloading SQLAlchemy-1.4.9-cp38-cp38-manylinux2014_x86_64.whl (1.5 MB)
Collecting pyyaml
 Downloading PyYAML-5.4.1-cp38-cp38-manylinux1_x86_64.whl (662 kB)
Collecting gitpython>=2.1.0
 Downloading GitPython-3.1.14-py3-none-any.whl (159 kB)
Collecting prometheus-flask-exporter
 Downloading prometheus_flask_exporter-0.18.1.tar.gz (21 kB)
Collecting docker>=4.0.0
 Downloading docker-5.0.0-py2.py3-none-any.whl (146 kB)
Collecting alembic<=1.4.1
 Downloading alembic-1.4.1.tar.gz (1.1 MB)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.19.2)
Collecting databricks-cli>=0.8.7
 Downloading databricks-cli-0.14.3.tar.gz (54 kB)
Requirement already satisfied: entrypoints in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.3)
Requirement already satisfied: pytz in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2020.5)
Collecting cloudpickle
 Downloading cloudpickle-1.6.0-py3-none-any.whl (23 kB)
Requirement already satisfied: click>=7.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-29616419-2237-4cad-81e8-dd77627eec76/lib/python3.8/site-packages (from mlflow) (7.1.2)
Collecting gunicorn; platform_system != "Windows"
 Downloading gunicorn-20.1.0.tar.gz (370 kB)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.1.3)
Collecting protobuf>=3.6.0
 Downloading protobuf-3.15.8-cp38-cp38-manylinux1_x86_64.whl (1.0 MB)
Collecting querystring-parser
 Downloading querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
Collecting sqlparse>=0.3.1
 Downloading sqlparse-0.4.1-py3-none-any.whl (42 kB)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.17.3->mlflow) (3.0.4)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.17.3->mlflow) (2.10)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.17.3->mlflow) (2020.12.5)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.17.3->mlflow) (1.25.11)
Collecting Jinja2>=2.10.1
 Downloading Jinja2-2.11.3-py2.py3-none-any.whl (125 kB)
Collecting itsdangerous>=0.24
 Downloading itsdangerous-1.1.0-py2.py3-none-any.whl (16 kB)
Collecting Werkzeug>=0.15
 Downloading Werkzeug-1.0.1-py2.py3-none-any.whl (298 kB)
Collecting greenlet!=0.4.17; python_version >= "3"
 Downloading greenlet-1.0.0-cp38-cp38-manylinux2010_x86_64.whl (165 kB)
Collecting gitdb<5,>=4.0.1
 Downloading gitdb-4.0.7-py3-none-any.whl (63 kB)
Collecting prometheus_client
 Downloading prometheus_client-0.10.1-py2.py3-none-any.whl (55 kB)
Collecting websocket-client>=0.32

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, IDF, HashingTF
from nltk.stem.snowball import SnowballStemmer
from pyspark.sql.types import StringType,ArrayType
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import hashlib
import sys
from pyspark.sql.types import IntegerType

In [0]:
train = spark.read.format("csv").load('/FileStore/tables/train.csv',HEADER=True)
test = spark.read.format("csv").load('/FileStore/tables/test.csv',HEADER=True)

In [0]:
train_label = spark.read.format("csv").load('/FileStore/tables/train_label.csv',HEADER=True)
test_label = spark.read.format("csv").load('/FileStore/tables/test_label.csv',HEADER=True)

In [0]:
train = train.join(train_label, on="Id", how="inner")
test = test.join(test_label, on="Id", how="inner")

train = train.withColumnRenamed("Category", "label")
test = test.withColumnRenamed("Category", "label")

from pyspark.sql.types import IntegerType
train = train.withColumn("label", train["label"].cast(IntegerType()))
test = test.withColumn("label", test["label"].cast(IntegerType()))

In [0]:
#dimension of test set
print((test.count(), len(test.columns)))

(54300, 5)

In [0]:
#dimension of train set
print((train.count(), len(test.columns)))

(217197, 5)

In [0]:
test.show()

+---+---+--------------------+------+-----+
 Id|_c0| description|gender|label|
+---+---+--------------------+------+-----+
 0| 3| She currently wo...| F| 6|
 1| 6| Lavalette’s phot...| M| 20|
 2| 11| Along with his a...| M| 24|
 3| 17|" She obtained he...| F| 19|
 4| 18| She studies issu...| F| 19|
 5| 34| He currently wri...| M| 6|
 6| 36| She found her wa...| F| 6|
 7| 42| He is also Presi...| M| 26|
 8| 44| His practice ser...| M| 22|
 9| 56| With Steven M. C...| M| 19|
 10| 59| Dr. Knight compl...| F| 19|
 11| 62| His photos have ...| M| 20|
 12| 65| Her interests ar...| F| 19|
 13| 71| She graduated fr...| F| 19|
 14| 73| Laurie earned a ...| F| 19|
 15| 83| He was lead coun...| M| 26|
 16| 94| He is a member o...| M| 19|
 17| 95| He also holds ho...| M| 19|
 18| 98| He earned a doct...| M| 16|
 19| 99| His research is ...| M| 19|
+---+---+--------------------+------+-----+
only showing top 20 rows

In [0]:
train.show()

+---+---+--------------------+------+-----+
 Id|_c0| description|gender|label|
+---+---+--------------------+------+-----+
 0| 0| She is also a Ro...| F| 19|
 1| 1| He is a member o...| M| 9|
 2| 2| Dr. Aster has he...| M| 19|
 3| 4| He runs a boutiq...| M| 24|
 4| 5| He focuses on cl...| M| 24|
 5| 7| He is author of ...| M| 22|
 6| 8| As an associate ...| F| 6|
 7| 9| He holds a Journ...| M| 6|
 8| 10| Her teachings ge...| F| 3|
 9| 12| For more quips a...| F| 14|
 10| 13| An elder in the ...| F| 19|
 11| 14| She has written ...| F| 19|
 12| 15| He is an expert ...| M| 19|
 13| 16| She wrote Black ...| F| 19|
 14| 19| She has expertis...| F| 19|
 15| 20| She grew up in P...| F| 26|
 16| 21| He is the author...| M| 6|
 17| 22| He is part of EF...| M| 26|
 18| 23| Her interests at...| F| 22|
 19| 24| He is the host o...| M| 19|
+---+---+--------------------+------+-----+
only showing top 20 rows

In [0]:
# Clean text
train_clean = train.select("Id", (lower(regexp_replace('description', "[^a-zA-Z\\s]", "")).alias('description')),"gender","label")

# Tokenize text
tokenizer2 = Tokenizer(inputCol='description', outputCol='words_token')
df_words_token2 = tokenizer2.transform(train_clean)

# Remove stop words
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
stemmed_words_train = remover.transform(df_words_token2)

# Stem text
stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens])
df_stemmed2 = stemmed_words_train.withColumn("words_stemmed", stemmer_udf("words_clean"))

# Filter length word > 3
filter_length_udf = udf(lambda row: [x for x in row if len(x) >= 3], ArrayType(StringType()))
document_cleaned_train = df_stemmed2.withColumn('words', filter_length_udf(col('words_stemmed')))

hashingTF = HashingTF(inputCol="words",outputCol='Term_frequency')
term_frequency_train = hashingTF.transform(document_cleaned_train)

In [0]:
# Clean text
test_clean = test.select("Id", (lower(regexp_replace('description', "[^a-zA-Z\\s]", "")).alias('description')),"gender", "label")

# Tokenize text
tokenizer2 = Tokenizer(inputCol='description', outputCol='words_token')
df_words_token2 = tokenizer2.transform(test_clean)

# Remove stop words
remover = StopWordsRemover(inputCol='words_token', outputCol='words_clean')
stemmed_words_train = remover.transform(df_words_token2)

# Stem text
stemmer = SnowballStemmer(language='english')
stemmer_udf = udf(lambda tokens: [stemmer.stem(token) for token in tokens])
df_stemmed2 = stemmed_words_train.withColumn("words_stemmed", stemmer_udf("words_clean"))

# Filter length word > 3
filter_length_udf = udf(lambda row: [x for x in row if len(x) >= 3], ArrayType(StringType()))
document_cleaned_test = df_stemmed2.withColumn('words', filter_length_udf(col('words_stemmed')))

hashingTF = HashingTF(inputCol="words",outputCol='Term_frequency')
term_frequency_test = hashingTF.transform(document_cleaned_test)

In [0]:
#term frequency vector for train set
term_frequency_train.show()

+---+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 Id| description|gender|label| words_token| words_clean| words_stemmed| words| Term_frequency|
+---+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 0| she is also a ro...| F| 19|[, she, is, also,...|[, also, ronald, ...|[, also, ronald, ...|[also, ronald, as...|(262144,[6392,898...|
 1| he is a member o...| M| 9|[, he, is, a, mem...|[, member, aicpa,...|[, member, aicpa,...|[member, aicpa, w...|(262144,[2586,253...|
 2| dr aster has hel...| M| 19|[, dr, aster, has...|[, dr, aster, hel...|[, dr, aster, hel...|[aster, held, tea...|(262144,[25453,37...|
 3| he runs a boutiq...| M| 24|[, he, runs, a, b...|[, runs, boutique...|[, run, boutiqu, ...|[run, boutiqu, de...|(262144,[15814,17...|
 4| he focuses on cl...| M| 24|[, he, focuses, o...|[, focuses, cloud...|[, focus, cloud, ...|[focus, cloud, se...|(262144,[1218,152...|
 5| he is author of ...| M| 22|[, he, is, author...|[, author, severa...|[, author, sever,...|[author, sever, b...|(262144,[8538,167...|
 6| as an associate ...| F| 6|[, as, an, associ...|[, associate, web...|[, associ, web, p...|[associ, web, pro...|(262144,[1512,207...|
 7| he holds a journ...| M| 6|[, he, holds, a, ...|[, holds, journal...|[, hold, journal,...|[hold, journal, m...|(262144,[1512,576...|
 8| her teachings ge...| F| 3|[, her, teachings...|[, teachings, get...|[, teach, get, st...|[teach, get, stra...|(262144,[39498,39...|
 9| for more quips a...| F| 14|[, for, more, qui...|[, quips, tips, r...|[, quip, tip, ref...|[quip, tip, refer...|(262144,[1512,207...|
 10| an elder in the ...| F| 19|[, an, elder, in,...|[, elder, united,...|[, elder, unit, m...|[elder, unit, met...|(262144,[25453,43...|
 11| she has written ...| F| 19|[, she, has, writ...|[, written, numer...|[, written, numer...|[written, numer, ...|(262144,[3176,567...|
 12| he is an expert ...| M| 19|[, he, is, an, ex...|[, expert, black,...|[, expert, black,...|[expert, black, p...|(262144,[12328,26...|
 13| she wrote black ...| F| 19|[, she, wrote, bl...|[, wrote, black, ...|[, wrote, black, ...|[wrote, black, ma...|(262144,[3176,332...|
 14| she has expertis...| F| 19|[, she, has, expe...|[, expertise, his...|[, expertis, hist...|[expertis, histor...|(262144,[19553,25...|
 15| she grew up in p...| F| 26|[, she, grew, up,...|[, grew, pakistan...|[, grew, pakistan...|[grew, pakistan, ...|(262144,[10723,28...|
 16| he is the author...| M| 6|[, he, is, the, a...|[, author, three,...|[, author, three,...|[author, three, b...|(262144,[29439,79...|
 17| he is part of ef...| M| 26|[, he, is, part, ...|[, part, effs, in...|[, part, eff, int...|[part, eff, intel...|(262144,[8804,119...|
 18| her interests at...| F| 22|[, her, interests...|[, interests, hel...|[, interest, help...|[interest, help, ...|(262144,[16420,39...|
 19| he is the host o...| M| 19|[, he, is, the, h...|[, host, skeptics...|[, host, skeptic,...|[host, skeptic, g...|(262144,[18919,20...|
+---+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
term_frequency_train = term_frequency_train.withColumnRenamed('Term_frequency', 'features')
term_frequency_test = term_frequency_test.withColumnRenamed('Term_frequency', 'features')

In [0]:
term_frequency_train.dtypes

Out[59]: [('Id', 'string'),
 ('description', 'string'),
 ('gender', 'string'),
 ('label', 'int'),
 ('words_token', 'array<string>'),
 ('words_clean', 'array<string>'),
 ('words_stemmed', 'string'),
 ('words', 'array<string>'),
 ('features', 'vector')]

In [0]:
term_frequency_train.show()

+---+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 Id| description|gender|label| words_token| words_clean| words_stemmed| words| features|
+---+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 0| she is also a ro...| F| 19|[, she, is, also,...|[, also, ronald, ...|[, also, ronald, ...|[also, ronald, as...|(262144,[6392,898...|
 1| he is a member o...| M| 9|[, he, is, a, mem...|[, member, aicpa,...|[, member, aicpa,...|[member, aicpa, w...|(262144,[2586,253...|
 2| dr aster has hel...| M| 19|[, dr, aster, has...|[, dr, aster, hel...|[, dr, aster, hel...|[aster, held, tea...|(262144,[25453,37...|
 3| he runs a boutiq...| M| 24|[, he, runs, a, b...|[, runs, boutique...|[, run, boutiqu, ...|[run, boutiqu, de...|(262144,[15814,17...|
 4| he focuses on cl...| M| 24|[, he, focuses, o...|[, focuses, cloud...|[, focus, cloud, ...|[focus, cloud, se...|(262144,[1218,152...|
 5| he is author of ...| M| 22|[, he, is, author...|[, author, severa...|[, author, sever,...|[author, sever, b...|(262144,[8538,167...|
 6| as an associate ...| F| 6|[, as, an, associ...|[, associate, web...|[, associ, web, p...|[associ, web, pro...|(262144,[1512,207...|
 7| he holds a journ...| M| 6|[, he, holds, a, ...|[, holds, journal...|[, hold, journal,...|[hold, journal, m...|(262144,[1512,576...|
 8| her teachings ge...| F| 3|[, her, teachings...|[, teachings, get...|[, teach, get, st...|[teach, get, stra...|(262144,[39498,39...|
 9| for more quips a...| F| 14|[, for, more, qui...|[, quips, tips, r...|[, quip, tip, ref...|[quip, tip, refer...|(262144,[1512,207...|
 10| an elder in the ...| F| 19|[, an, elder, in,...|[, elder, united,...|[, elder, unit, m...|[elder, unit, met...|(262144,[25453,43...|
 11| she has written ...| F| 19|[, she, has, writ...|[, written, numer...|[, written, numer...|[written, numer, ...|(262144,[3176,567...|
 12| he is an expert ...| M| 19|[, he, is, an, ex...|[, expert, black,...|[, expert, black,...|[expert, black, p...|(262144,[12328,26...|
 13| she wrote black ...| F| 19|[, she, wrote, bl...|[, wrote, black, ...|[, wrote, black, ...|[wrote, black, ma...|(262144,[3176,332...|
 14| she has expertis...| F| 19|[, she, has, expe...|[, expertise, his...|[, expertis, hist...|[expertis, histor...|(262144,[19553,25...|
 15| she grew up in p...| F| 26|[, she, grew, up,...|[, grew, pakistan...|[, grew, pakistan...|[grew, pakistan, ...|(262144,[10723,28...|
 16| he is the author...| M| 6|[, he, is, the, a...|[, author, three,...|[, author, three,...|[author, three, b...|(262144,[29439,79...|
 17| he is part of ef...| M| 26|[, he, is, part, ...|[, part, effs, in...|[, part, eff, int...|[part, eff, intel...|(262144,[8804,119...|
 18| her interests at...| F| 22|[, her, interests...|[, interests, hel...|[, interest, help...|[interest, help, ...|(262144,[16420,39...|
 19| he is the host o...| M| 19|[, he, is, the, h...|[, host, skeptics...|[, host, skeptic,...|[host, skeptic, g...|(262144,[18919,20...|
+---+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
#First Model : Logistic Regression Classifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np
#lr = LogisticRegression(maxIter = 10)

#paramGrid_lr = ParamGridBuilder() \
#    .addGrid(lr.regParam, np.linspace(0.3, 0.01, 10)) \
#    .addGrid(lr.elasticNetParam, np.linspace(0.3, 0.8, 6)) \
#    .build()
#crossval_lr = CrossValidator(estimator=lr,
#                          estimatorParamMaps=paramGrid_lr,
#                          evaluator=BinaryClassificationEvaluator(),
#                          numFolds= 5)  
#cvModel_lr = crossval_lr.fit(term_frequency_train)
#best_model_lr = cvModel_lr.bestModel.summary
#best_model_lr.predictions.columns

In [0]:
#fit logistic regression
lr = LogisticRegression(maxIter = 10)
model = lr.fit(term_frequency_train)

In [0]:
#prediction
predictions = model.transform(term_frequency_test)

In [0]:
#show prediction
predictions.select("prediction").show()

+----------+
prediction|
+----------+
 6.0|
 20.0|
 19.0|
 19.0|
 19.0|
 6.0|
 26.0|
 26.0|
 11.0|
 19.0|
 19.0|
 20.0|
 19.0|
 19.0|
 19.0|
 19.0|
 26.0|
 19.0|
 19.0|
 19.0|
+----------+
only showing top 20 rows

In [0]:
#get F1 score
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_lr.evaluate(predictions)

Out[66]: 0.7123838210798265